In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# import import_ipynb
# from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Humidity Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 10


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609
1,TANGARA_2B42,d29e6pg,D29ESP32DED2B42,3.4270477294921875 -76.54792785644531,3.427048,-76.547928
2,TANGARA_2E9A,d29ed62,D29ESP32DED2E9A,3.4407806396484375 -76.54106140136719,3.440781,-76.541061
3,TANGARA_2FF6,d29e66v,D29ESP32DED2FF6,3.3995819091796875 -76.53419494628906,3.399582,-76.534195
4,TANGARA_307A,d29e48s,D29ESP32DED307A,3.3432769775390625 -76.52458190917969,3.343277,-76.524582


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-10-30T23:00:00-05:00 -> 1698724800000, Until: 2023-10-31T00:00:00-05:00 -> 1698728400000
Group by Time: 30s


In [4]:
# Humidity Data Frame Sensors
df_hum_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'hum', GROUP_BY_TIME)
df_hum_raw.head()
#df_hum_raw.shape

,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,
2023-10-30 23:00:00-05:00,78.93,75.03,66.68,82.39,72.22,78.25,99.9,99.9,68.85,74.69
2023-10-30 23:00:30-05:00,78.84,75.31,67.01,82.61,72.24,78.17,99.9,99.9,68.86,74.88
2023-10-30 23:01:00-05:00,78.74,75.45,67.16,82.73,72.10,78.35,99.9,99.9,69.01,74.53
2023-10-30 23:01:30-05:00,78.54,75.37,66.86,83.23,72.07,78.31,99.9,99.9,69.17,75.44
2023-10-30 23:02:00-05:00,78.28,75.35,67.22,81.98,72.34,78.16,99.9,99.9,69.28,74.54


## Descriptive Statistics

In [5]:
# Describe Data
df_hum_raw.describe()

,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,116.000000,113.000000,115.000000,115.000000,111.000000,115.000000,1.120000e+02,1.160000e+02,116.000000,116.000000
mean,78.311724,76.418230,67.487478,82.572522,72.702342,79.457739,9.990000e+01,9.990000e+01,68.998448,76.956983
std,0.503259,0.455877,0.414462,0.361034,0.426887,0.712795,1.427472e-14,2.854501e-14,0.390884,0.824826
min,77.130000,75.030000,66.580000,81.610000,72.010000,78.160000,9.990000e+01,9.990000e+01,68.000000,74.530000
25%,77.970000,76.250000,67.220000,82.380000,72.245000,78.780000,9.990000e+01,9.990000e+01,68.810000,76.710000
50%,78.300000,76.570000,67.390000,82.610000,72.800000,79.700000,9.990000e+01,9.990000e+01,69.110000,77.265000
75%,78.595000,76.680000,67.820000,82.780000,73.020000,80.080000,9.990000e+01,9.990000e+01,69.272500,77.482500
max,79.560000,77.160000,68.670000,83.800000,73.490000,80.600000,9.990000e+01,9.990000e+01,69.530000,77.990000


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_hum_raw.columns:
    df_hum_sensor = df_hum_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_hum_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%


/home/sebaxtian/Workspaces/Tangara/tangara-pipeline/standalone/notebooks/utils.ipynb:197: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  missing_data_percent = round(df_sensor.isna().sum()[0] * 100 / total)
/home/sebaxtian/Workspaces/Tangara/tangara-pipeline/standalone/notebooks/utils.ipynb:198: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_percent = round(df_sensor.count()[0] * 100 / total)
/home/sebaxtian/Workspaces/Tangara/tangara-pipeline/standalone/notebooks/utils.ipynb:197: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_hum_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_hum_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_hum_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_hum_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save Humidity Data Frame Sensors into CSV file
df_to_csv(df_hum_raw, "hum_raw.csv")